<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [2]</a>'.</span>

# PA005: High Value Customer Identification (Insiders)

# <div style="color:red;display:fill;border-radius:15px;background-color:#5d6d7e;letter-spacing:0.2px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:100%">0.0. Imports</p></div>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import inflection
import umap.umap_ as umap
import re
import s3fs

import matplotlib.cm as cm
from matplotlib        import pyplot as plt
from plotly            import express as px

from sklearn           import cluster as c
from sklearn           import metrics as mt
from sklearn           import ensemble as en
from sklearn           import preprocessing as pp
from sklearn           import decomposition as dd
from sklearn           import mixture as mx
from sklearn.manifold  import TSNE
from sklearn.neighbors import NearestNeighbors

from scipy.cluster     import hierarchy as hc

import warnings
warnings.filterwarnings( 'ignore' )

/home/luanmaieski/.pyenv/versions/3.11.4/envs/pa005clustering/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/luanmaieski/.pyenv/versions/3.11.4/envs/pa005clustering/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/luanmaiesk

/home/luanmaieski/.pyenv/versions/3.11.4/envs/pa005clustering/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/luanmaieski/.pyenv/versions/3.11.4/envs/pa005clustering/lib/python3.11/site-packages/umap/umap_.py:660: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()


## 0.2. Load Dataset

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [2]:
# load data
path_local = '/home/luanmaieski/projetos/insiders_clustering/'
path_s3 = 's3://insidersdatasetlgm/'
df_raw = pd.read_csv( path_s3 + 'Ecommerce.csv', encoding='iso-8859-1')

df_raw.head()
# drop extra column
#df_raw = df_raw.drop(columns=['Unnamed: 8'], axis=1 )

PermissionError: Forbidden

In [ ]:
df_raw.head()

# <div style="color:red;display:fill;border-radius:15px;background-color:#5d6d7e;letter-spacing:0.2px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:100%">1.0. Data Description</p></div>

In [ ]:
df1 = df_raw.copy()

## 1.1. Rename Columns

In [ ]:
df1.columns

In [ ]:
cols_old = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate','UnitPrice', 'CustomerID', 'Country']

snakecase = lambda x: inflection.underscore( x )

cols_new = list(map( snakecase, cols_old ))

#rename
df1.columns = cols_new

In [ ]:
df1.sample()

## 1.2. Data dimensions

In [ ]:
print( 'Número de Linhas: {}\nNúmero de Colunas: {}'.format(df1.shape[0], df1.shape[1]) )

## 1.3. Data types

In [ ]:
df1.dtypes

## 1.4. Check NA

In [ ]:
df1.isna().sum()

## 1.5. Replace NA

In [ ]:
df_missing = df1.loc[df1['customer_id'].isna(), :]
df_not_missing = df1.loc[~df1['customer_id'].isna(), :]

In [ ]:
# create reference
df_backup = pd.DataFrame( df_missing['invoice_no'].drop_duplicates() )
df_backup['customer_id'] = np.arange( 19000, 19000+len( df_backup), 1)

# merge original with reference dataframe
df1 = pd.merge( df1, df_backup, on='invoice_no', how='left' )

# coalesce
df1['customer_id'] = df1['customer_id_x'].combine_first( df1['customer_id_y'] )

# drop extra columns
df1 = df1.drop( columns=['customer_id_x', 'customer_id_y'], axis=1)

df1.head()

In [ ]:
df1.isna().sum()

##  1.6 Change types

In [ ]:
# invoice date
df1['invoice_date'] = pd.to_datetime( df1['invoice_date'], format='%d-%b-%y' )

# customer id
df1['customer_id'] = df1['customer_id'].astype( 'int64' )

In [ ]:
df1.dtypes

## 1.7. Descriptive Statistics

In [ ]:
num_attributes = df1.select_dtypes( include=['int64', 'float64'] )
cat_attributes = df1.select_dtypes( exclude=['int64', 'float64', 'datetime64[ns]'] )

### 1.7.1. Numerical Attributes

In [ ]:
# central tendency - mean, median
ct1 = pd.DataFrame( num_attributes.apply( np.mean ) ).T
ct2 = pd.DataFrame( num_attributes.apply(np.median ) ).T

# dispersion - desvio padrão, minimo, maximo, range, skew, kurtosis
d1 = pd.DataFrame( num_attributes.apply(np.std ) ).T
d2 = pd.DataFrame( num_attributes.apply(np.min ) ).T
d3 = pd.DataFrame( num_attributes.apply(np.max ) ).T
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew()) ).T
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T

# concatenate
m = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6] ).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

#### 1.7.1.1. Numerical Attributes - Investigating

1. Quantity negativa ( pode ser devolução )

2. Preço unitário igual a zero ( pode ser promoção? )

### 1.7.2. Categorical Attributes

#### Invoice number

In [ ]:
# problema: Temos invoce com letras e numeros
# identificação:
df_letter_invoices = df1.loc[df1['invoice_no'].apply( lambda x: bool( re.search( '[^0-9]+', x ) ) ), :]

print( 'Total number of invoices: {}'.format( len( df_letter_invoices ) ) )
print( 'Total number of negative quantity: {}'.format( len( df_letter_invoices[df_letter_invoices['quantity'] < 0] ) ) )

#### Stock code

In [ ]:
# check stock codes only characters
df1.loc[df1['stock_code'].apply( lambda x: bool( re.search( '^[a-zA-Z]+$', x ) ) ), 'stock_code'].unique()

# Ação:
# 1. Remove stock_code in ['POST', 'D', 'M', 'PADS', 'DOT', 'CRUK']

#### Description

In [ ]:
# Ação: Delete description

# <div style="color:red;display:fill;border-radius:15px;background-color:#5d6d7e;letter-spacing:0.2px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:100%">2.0. Variable Filters</p></div>  

In [ ]:
df2 = df1.copy()

In [ ]:
# ===== Numerical Attributes =====
# unit price > 0.0
df2 = df2.loc[df2['unit_price'] > 0.04, :]

# ==== Categorical attributes ====
# stock code != [POST, D, M, DOT, CRUK]
df2 = df2[~df2['stock_code'].isin( ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK'] ) ]

# description
df2 = df2.drop( columns='description', axis=1 )

# map
df2 = df2[~df2['country'].isin( ['European Community', 'Unspecified'] ) ]

# bad users
df2 = df2[~df2['customer_id'].isin( [16446]) ]

# quantity
df2_returns = df2.loc[df2['quantity'] < 0, :]
df2_purchase = df2.loc[df2['quantity'] > 0, :]

# <div style="color:red;display:fill;border-radius:15px;background-color:#5d6d7e;letter-spacing:0.2px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:100%">3.0. Feature Engineering</p></div> 

In [ ]:
df3 = df2.copy()

## 3.1. Feature Creation

In [ ]:
# data reference
df_ref = df2.drop(['invoice_no', 'stock_code', 'quantity', 'invoice_date', 'unit_price', 'country'], axis=1).drop_duplicates( ignore_index=True )

In [ ]:
df_ref.shape

### 3.1.1. Gross Revenue

In [ ]:
# Gross Revenue ( Faturamento ) quantity * price
df2_purchase.loc[:, 'gross_revenue'] = df2_purchase.loc[:, 'quantity'] * df2_purchase.loc[:, 'unit_price']

# Monetary
df_monetary = df2_purchase[['customer_id', 'gross_revenue']].groupby( 'customer_id').sum().reset_index()
df_ref = pd.merge( df_ref, df_monetary, on='customer_id', how='left' )
df_ref.isna().sum()

### 3.1.2. Recency - Day from last purchase

In [ ]:
# Recency - Last day purchase
df_recency = df2_purchase[['customer_id', 'invoice_date']].groupby( 'customer_id' ).max().reset_index()
df_recency['recency_days'] = (df2_purchase['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()
df_ref = pd.merge( df_ref, df_recency, on='customer_id', how='left' )
df_ref.isna().sum()

### 3.1.5 Quantity of products purchased

In [ ]:
# Número de produtos 
df_freq = ( df2_purchase.loc[:, ['customer_id', 'stock_code']].groupby( 'customer_id' ).count()
                                                              .reset_index()
                                                              .rename( columns={'stock_code': 'qtde_products'}) )
df_ref = pd.merge( df_ref, df_freq, on='customer_id', how='left' )
df_ref.isna().sum()

### 3.1.7. Frequency Purchase

In [ ]:
df_aux = ( df2_purchase[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                             .groupby('customer_id')
                                                             .agg( max_ = ('invoice_date', 'max'),
                                                                   min_ = ('invoice_date', 'min'),
                                                                   days_ = ('invoice_date', lambda x: (( x.max() - x.min() ).days) + 1),
                                                                   buy_ = ('invoice_no', 'count') ) ).reset_index()

# Frequency
df_aux['frequency'] = df_aux[['buy_', 'days_']].apply( lambda x: x['buy_'] / x['days_'] if x['days_'] != 0 else 0, axis=1 )

# merge
df_ref = pd.merge( df_ref, df_aux[['customer_id', 'frequency']], on='customer_id', how='left' )
df_ref.isna().sum()

### 3.1.8. Number of Returns

In [ ]:
# Number of Returns
df_returns = df2_returns[['customer_id', 'quantity']].groupby( 'customer_id' ).sum().reset_index().rename( columns={'quantity': 'qtde_returns'} )
df_returns['qtde_returns'] = df_returns['qtde_returns'] * -1

df_ref = pd.merge( df_ref, df_returns, how='left', on='customer_id' )
df_ref.loc[df_ref['qtde_returns'].isna(), 'qtde_returns'] = 0 

df_ref.isna().sum()

# <div style="color:red;display:fill;border-radius:15px;background-color:#5d6d7e;letter-spacing:0.2px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:100%">4.0. EDA (Exploratory Data Analysis)</p></div>   

In [ ]:
df4 = df_ref.dropna().copy()

In [ ]:
len(df4['customer_id'].unique())

## 4.3. Estudo do Espaço

In [ ]:
# selected dataset
cols_selected = ['customer_id', 'gross_revenue', 'recency_days', 'qtde_products', 'frequency', 'qtde_returns']
df43 = df4[cols_selected].drop( columns='customer_id', axis=1 )

In [ ]:
df43.head()

In [ ]:
mm = pp.MinMaxScaler()

df43['gross_revenue'] = mm.fit_transform( df43[['gross_revenue']] )
df43['recency_days'] = mm.fit_transform( df43[['recency_days']] )
df43['qtde_products'] = mm.fit_transform( df43[['qtde_products']] )
df43['frequency'] = mm.fit_transform( df43[['frequency']] )
df43['qtde_returns'] = mm.fit_transform( df43[['qtde_returns']] )

### 4.3.3. Tree-Based Embedding

In [ ]:
# training dataset
X = df43.drop( columns=['gross_revenue'], axis=1 )
y = df43['gross_revenue']

# model definition
rf_model = en.RandomForestRegressor( n_estimators=100, random_state=42 )

# model training
rf_model.fit( X, y )

# leaf
df_leaf = pd.DataFrame( rf_model.apply( X ) )

In [ ]:
# reduzer dimensionality
reducer = umap.UMAP( random_state=42 )
embedding = reducer.fit_transform( df_leaf )

# embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:, 0]
df_tree['embedding_y'] = embedding[:, 1]

# plot UMAP
sns.scatterplot( x='embedding_x',
                 y='embedding_y', 
                 data=df_tree )

# <div style="color:red;display:fill;border-radius:15px;background-color:#5d6d7e;letter-spacing:0.2px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:100%">5.0. Data Preparation</p></div>      

In [ ]:
df5 = df_tree.copy()
df5.to_csv( '../src/data/tree_based_embedding.csv', index=False ) 

# <div style="color:red;display:fill;border-radius:15px;background-color:#5d6d7e;letter-spacing:0.2px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:100%">7.0. Hiperparameter Fine-Tunning</p></div>      

In [ ]:
X = df5.copy()

In [ ]:
X.head()

# <div style="color:red;display:fill;border-radius:15px;background-color:#5d6d7e;letter-spacing:0.2px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:100%">8.0. Model Training</p></div>       

## 8.1. K-Means

In [ ]:
# model definition
k = 8
kmeans = c.KMeans( init='random', n_clusters=k, n_init=100, max_iter=300, random_state=42 )

# model training
kmeans.fit( X )

# clustering 
labels = kmeans.labels_

## 8.2. Cluster Validation

In [ ]:
## WSS ( Within-Cluster Sum of Square )
print( 'WSS value: {}'.format(kmeans.inertia_ ) )

## SS ( Silhouette Score )
print( 'SS value: {}'.format( mt.silhouette_score( X, labels, metric='euclidean' ) ) )

# <div style="color:red;display:fill;border-radius:15px;background-color:#5d6d7e;letter-spacing:0.2px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:100%">9.0. Cluster Analysis</p></div>        

In [ ]:
df9 = X.copy()
df9['cluster'] = labels

In [ ]:
#df92 = df4.copy()
#df9['cluster'] = labels
#df9.head()

## 9.1. Visualization Inspection

In [ ]:
sns.scatterplot( x='embedding_x', y='embedding_y', data=df9, hue='cluster', palette='deep')

In [ ]:
#visualizer = SilhouetteVisualizer( kmeans, colors='yellowbrick' )
#visualizer.fit( X )
#visualizer.finalize()

In [ ]:
#fig = px.scatter_3d( df9, x='recency_days', y='invoice_no', z='gross_revenue', color='cluster' )
#fig.show()

## 9.2. 2d plot

In [ ]:
df_viz = df9.drop( columns='customer_id', axis=1 )
sns.pairplot( df_viz, hue='cluster')

## 9.3. UMAP 

In [ ]:
df_viz = df9.drop( columns='customer_id', axis=1 )

In [ ]:
reducer = umap.UMAP( n_neighbors=35, random_state=42 )
embedding = reducer.fit_transform( X )

# embedding
df_viz['embedding_x'] = embedding[:, 0]
df_viz['embedding_y'] = embedding[:, 1]

# plot UMAP 
plt.figure(figsize=(10, 6))
sns.scatterplot( x='embedding_x', y='embedding_y', 
                 hue='cluster',
                 palette=sns.color_palette( 'hls', n_colors=len( df_viz['cluster'].unique() ) ),
                 data=df_viz )
plt.show()

## 9.2. Cluster Profile

In [ ]:
df92 = df4[cols_selected].copy()
df92['cluster'] = labels
df92.head()

# change dtypes
df92['recency_days'] = df92['recency_days'].astype( 'int64')
df92['qtde_products'] = df92['qtde_products'].astype( 'int64')
df92['qtde_returns'] = df92['qtde_returns'].astype( 'int64')

In [ ]:
# Number of customer
df_cluster = df92[['customer_id', 'cluster']].groupby( 'cluster' ).count().reset_index()
df_cluster['perc_customer'] = 100*( df_cluster['customer_id'] / df_cluster['customer_id'].sum() )

# Avg Gross revenue
df_avg_gross_revenue = df92[['gross_revenue', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_gross_revenue, how='inner', on='cluster' )

# Avg recency days
df_avg_recency_days = df92[['recency_days', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_recency_days, how='inner', on='cluster')

# Avg invoice_no
df_qtde_products = df92[['qtde_products', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge( df_cluster, df_qtde_products, how='inner', on='cluster' )

# Frequency
df_frequency = df92[['frequency', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge( df_cluster, df_frequency, how='inner', on='cluster' )

# Returns
df_qtde_returns = df92[['qtde_returns', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge( df_cluster, df_qtde_returns, how='inner', on='cluster' )

df_cluster

In [ ]:
1 Cluster Insiders
5 Cluster More Products
0 Cluster Stop Returns
7 Cluster More Frequency
6 Cluster Spend Money
4 Cluster Less Days
2 Cluster Less 1k
3 Cluster More Buy

### Cluster 01: (Candidato à Insider)
    - Numero de customers: 468 (16% dos customers)
    - Faturamento médio: 8835
    - Recência média: 21 dias
    - Média de produtos comprados: 424
    - Frequência do Produtos comprados: 0.09 produtos/dia
    - Receita em média: 8835,89 dólares

# <div style="color:red;display:fill;border-radius:15px;background-color:#5d6d7e;letter-spacing:0.2px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:100%">10.0. Análise Exploratória de Dados</p></div>         

In [ ]:
df10 = df92.copy()

In [ ]:
df10.head()

## 10.1. MindMap de hipóteses

## 10.2. Hipóteses de Negócio

In [ ]:
Cluster | Variável para testar | Base de Comparação

### Hipótese Compra

1. Os clientes do cluster insiders usam cartão de crédito em 80% das compras.
2. Os clientes do cluster insiders possuem um ticket médio de 10% acima do cluster More Products
3. Os clientes do cluster insiders possuem um basket size acima de 5 produtos.
4. **Os clientes do cluster insiders possuem um volume (produtos) de compras acima de 10% do total de compras.**
5. **Os clientes do cluster insiders possuem um volume (faturamento) de compras acima de 10% do total de compras.**
6. **Os clientes do cluster insiders tem um número de devolução abaixo da média da base total de clientes.**
7. **A mediana do faturamento pelos clientes do cluster insiders é 10% maior do que a mediana dp faturamento geral.**
8. **O percentil do preço dos produtos comprados pelos clientes insiders.**

### H1: Os clientes do cluster insiders possuem um volume (produtos) de compras acima de 10% do total de compras.
**Verdade:** Os cluster insiders possuem um volume de compra de produtos de 54%

In [ ]:
# soma da qtde de produtos do insiders
df_sales_insiders = df10.loc[df10['cluster'] == 1, 'qtde_products'].sum()
df_sales_total = df10.loc[:, 'qtde_products'].sum()

# soma da qtde de produtos total
print( '% Sales Insiders: {:.2f}%'.format( 100*df_sales_insiders/df_sales_total ) )

### H2: Os clientes do cluster insiders possuem um volume (faturamento) de compras acima de 10% do total de compras.
**Verdadeiro:** O cluster insiders possuem um volume de GMV de 52%

In [ ]:
# soma da qtde de produtos do insiders
df_gmv_insiders = df10.loc[df10['cluster'] == 1, 'gross_revenue'].sum()
df_gmv_total = df10.loc[:, 'gross_revenue'].sum()

# soma da qtde de produtos total
print( '% GMV Insiders: {:.2f}%'.format( 100*df_gmv_insiders/df_gmv_total ) )

### H3: Os clientes do cluster insiders tem um número de devolução abaixo da média da base total de clientes.
**Falso** O cluster insiders tem média de devoluções acima da média geral

In [ ]:
# média de devolução do cluster insider
df_avg_return_insiders = df10.loc[df10['cluster'] == 1, 'qtde_returns'].mean()

# média de devolução do total
df_avg_return_all = df10['qtde_returns'].mean()

print( 'Avg Return Insiders: {} vs Avg Return All: {}'.format( np.round( df_avg_return_insiders, 0 ), np.round( df_avg_return_all, 0 ) ) )

### H4: A mediana do faturamento pelos clientes do cluster insiders é 10% maior do que a mediana dp faturamento geral.
**Verdadeiro** A média do faturamento é 276% acima da média.

In [ ]:
# mediana de gmv do insiders
df_median_gmv_insiders = df10.loc[df10['cluster'] == 1, 'gross_revenue'].median()

# mediana de gmv do total
df_median_gmv_total = df10.loc[:, 'gross_revenue'].median()

gmv_diff = ( df_median_gmv_insiders - df_median_gmv_total ) / df_median_gmv_total
print( 'Median Diff: {:.2f}%'.format( 100*gmv_diff ) )

### H5: O gmv dos clientes insiders esta concentrados no 3º quartil.
**Falso** O faturamento do cluster insiders está concentrado no quartil 1

In [ ]:
df_aux = df10.loc[(df10['cluster'] == 1) & (df10['gross_revenue'] < 10000 ), 'gross_revenue'];
sns.violinplot( x=df_aux ) 

### Hipótese Cliente

1. 60% dos clientes do cluster insiders possuem o estado civil de solteiro.
2. 10% dos clientes do cluster insiders estão na faixa de 24-35 anos.
3. 40% das localidades de entrega do cluster insiders estão dentro de um raio de 50km.
4. 5% dos clientes do cluster insiders recebem mais de 100 mil dólares anualmente.
5. 90% dos clientes do cluster insiders tem ensino superior completo.

### Hipótese Produto

1. 30% de todos os produtos em pacotes grandes são comprados pelos clientes do cluster insiders.
2. A mediana dos preços dos produtos comprados pelos clientes do cluster insiders é 10% maior do que a mediana de todos os preços dos produtos.
3. O percentil do preço dos produtos comprados pelos clientes insiders.
4. O peso médio dos produtos comprados pelos clientes do cluster insiders é maior do que o peso médio dos outros clusters.
5. A idade média dos produtos comprados pelos clientes do cluster insiders é menor do que 15 dias.

### Perguntas de Negócio

### 1. Quem são as pessoas elegíveis para participar do programa de Insiders?

In [ ]:
df10.loc[df10['cluster'] == 1, 'customer_id'].head()

### 2. Quantos clientes farão parte do grupo?

In [ ]:
df10.loc[df10['cluster'] == 1, 'customer_id'].size

### 3. Quais as principais características desses clientes?

#### Cluster 01: (Candidato à Insider)
    - Numero de customers: 468 (16% dos customers)
    - Faturamento médio: 8835
    - Recência média: 21 dias
    - Média de produtos comprados: 424
    - Frequência do Produtos comprados: 0.09 produtos/dia
    - Receita em média: 8835,89 dólares

### 4. Qual a porcentagem de contribuição do faturamento, vinda do Insiders?

In [ ]:
df_insiders_gmv = df10.loc[df10['cluster'] == 1, 'gross_revenue'].sum()
df_all_gmv = df10.loc[:, 'gross_revenue'].sum()

print( '% GMV from Insiders: {}'.format( df_insiders_gmv / df_all_gmv ) )

### 5. Qual a expectativa de faturamento desse grupo para os próximos meses?

### 6. Quais as condições para uma pessoa ser elegível ao Insiders?

### 7. Quais as condições para uma pessoa ser removida do Insiders?

8. Qual a garantia que o programa Insiders é melhor que o restante da base?

9. Quais ações o time de marketing pode realizar para aumentar o faturamento?

## 10.3. Priorização das Hipótese

## 10.4 Validação das Hipóteses

## 10.5. Quadro de Respostas

# <div style="color:red;display:fill;border-radius:15px;background-color:#5d6d7e;letter-spacing:0.2px;overflow:hidden"><p style="padding:10px;color:white;overflow:hidden;text-align: center;margin:0;font-size:100%">11.0. Deploy to Production</p></div>         

In [ ]:
import sqlite3
from sqlalchemy import create_engine

In [ ]:
df92.dtypes

## 11.1. Insert into SQLITE

In [ ]:
# database conection
conn = create_engine( 'sqlite:///insiders_db.sqlite' )

## drop table
#query_drop_insiders = """
#    DROP TABLE insiders
#"""

# create table
query_create_insiders = """
    CREATE TABLE insiders (
        customer_id      INTEGER,
        gross_revenue    REAL,
        recency_days     INTEGER,
        qtde_products    INTEGER,
        frequency        REAL,
        qtde_returns     INTEGER,
        cluster          INTEGER  
    
    )
"""

conn.execute( query_create_insiders )

In [ ]:
df92.to_sql ( 'insiders', con=conn, if_exists='append', index=False )

In [ ]:
## get query
#query_collect = """
#    SELECT * FROM insiders"""
#
#df = pd.read_sql_query( query_collect, conn )
#df.head()

In [ ]:
pwd